In this notebook we build a classifier that takes features and classifies Faraday complexity.

In [72]:
import numpy, matplotlib.pyplot as plt
import torch, torch.nn, torch.optim
from tqdm import tqdm_notebook as tqdm
import sklearn.model_selection

import torchsample  # https://github.com/ncullen93/torchsample.git

In [21]:
features = {}
for name in ['e2e', 'pca', 'truth', 'dirty', 'clean', 'recon']:
    features[name] = torch.from_numpy(numpy.load('encoded_features_{}.npy'.format(name)))
    
labels = numpy.load('labels.npy')
torch_labels = torch.from_numpy(numpy.stack([labels, 1 - labels]).T).float()

In [56]:
class FaradayComplexityClassifier(torch.nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.n_features = n_features
        
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(n_features, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 2),
            torch.nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        return self.layers(x)

In [73]:
train_Xi, test_Xi = sklearn.model_selection.train_test_split(range(len(labels)), test_size=0.2)
train_Xi, val_Xi = sklearn.model_selection.train_test_split(train_Xi, test_size=0.2)

In [76]:
def train_nn(name, n_epochs):
    train_X = features[name][train_Xi]
    train_Y = torch_labels[train_Xi]
    val_X = features[name][val_Xi]
    val_Y = torch_labels[val_Xi]
    
    nn = FaradayComplexityClassifier(features[name].shape[1])
    trainer = torchsample.modules.ModuleTrainer(nn)
    trainer.compile(loss='bceloss',
                    optimizer='adam')
    trainer.fit(train_X, train_Y,
                val_data=(val_X, val_Y),
                num_epoch=n_epochs,
                batch_size=128,
                verbose=1)
    return nn

#     opt = torch.optim.Adam(nn.parameters(), lr=1e-3)
#     loss_fn = torch.nn.BCELoss()

#     bar = tqdm(total=n_epochs)
#     for e in range(n_epochs):
#         opt.zero_grad()
#         res = nn(train_X)
#         loss = loss_fn(res, train_Y)
#         loss.backward()
#         opt.step()

#         accuracy = (res.argmax(dim=1) == train_Y.argmax(dim=1)).float().mean()
#         bar.postfix = '{:0.1%}'.format(accuracy.item())
#         bar.update(1)
#     return nn

In [77]:
nns = {}
for name in ['e2e', 'pca', 'truth', 'dirty', 'clean', 'recon']:
    print(name)
    nns[name] = train_nn(name, 500)

e2e


ValueError: Invalid loss string input - must match pytorch function.

In [78]:
nn_e2e = train_nn('e2e')

TypeError: train_nn() missing 1 required positional argument: 'n_epochs'